In [107]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import tensorflow as tf

In [115]:
np.random.seed(125)
#Create one hot encoded class variables for 101 classes
all_classes = tf.one_hot(list(range(1,102)),101)

In [2]:
#Create folder for storing the frames
 
splitfiledir = r"..\ucfTrainTestlist"
splitfile = "mytest.txt"
splitname = splitfile.split('.')[0]

In [3]:
def storeFramesAndFlows(framesdir,splitfiledir,splitfile,splitname):
    
    #Read splitfile
    lines = open(os.path.join(splitfiledir,splitfile),"r")
    for line in lines:
        arr = line.split(" ")
        vidclass = arr[1] 
        line = arr[0].split("/")
        print (line)
        action = line[0]
        filename = line[1]
        actionpath = os.path.join(framesdir,splitname,action)
        framepath = os.path.join(actionpath,filename,"frames")
        flowpath = os.path.join(actionpath,filename,"flows")
        
        #Create folder for Action
        if not os.path.exists(actionpath):
            os.mkdir(actionpath)
        
        #Create folder for videofile
        if not os.path.exists(os.path.join(actionpath,filename)):
            os.mkdir(os.path.join(actionpath,filename))
        
        #Create folder for frames
        if not os.path.exists(framepath):
            os.mkdir(framepath)
            
        #Create folder for flows
        if not os.path.exists(flowpath):
            os.mkdir(flowpath)

        #Read video and collect frames, flows
        vidcap = cv2.VideoCapture(os.path.join(path,action,filename))
        count = 0 
        prevFrame = None
        nextFrame = None
        while True:
            success,image = vidcap.read()
            if not success:
                break
            frame = "frame_%d.jpg"%count
            flow_x = "flow_x_%d.jpg"%count
            flow_y = "flow_y_%d.jpg"%count
            framename = os.path.join(framepath,frame)
            flowname_x = os.path.join(flowpath,flow_x)
            flowname_y = os.path.join(flowpath,flow_y)
            if count == 0:
                prevFrame = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
                cv2.imwrite(framename,image)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                count += 1
                continue
            
            nextFrame = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
            cv2.imwrite(framename,image)
            optical_flow = cv2.optflow.DualTVL1OpticalFlow_create()
            flow = optical_flow.calc(prevFrame, nextFrame, None)
            prevFrame = nextFrame
            flow[...,0] = cv2.normalize(flow[...,0],None,0,255,cv2.NORM_MINMAX)
            flow[...,1] = cv2.normalize(flow[...,1],None,0,255,cv2.NORM_MINMAX)
            cv2.imwrite(flowname_x,flow[...,0])
            cv2.imwrite(flowname_y,flow[...,1])

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            count += 1
            
        filename = os.path.join(actionpath,filename,"info.txt")
        #Store the frames count in txt file
        rate = open(filename,"w")
        rate.write("frames:"+str(count))
        rate.write("\n")
        rate.write("class:"+vidclass)
        rate.close()

        #Close the video object
        vidcap.release()

    print ("Stored")

In [4]:
#Store Frames
framesdir = r"..\FramesFlows"

path = r"E:\capstone_adbi_data\UCF-101"


if not os.path.exists(os.path.join(framesdir,splitname)):
    os.mkdir(os.path.join(framesdir,splitname))
    storeFramesAndFlows(framesdir,splitfiledir,splitfile,splitname)
else:
    print ("Frames stored already!")

Frames stored already!


In [8]:
#Decided to keep number of segments as 5 for now

In [112]:
#Get the class
infopath = r"..\FramesFlows\mytest\ApplyLipstick\v_ApplyLipstick_g01_c01.avi\info.txt"
f = open(infopath,"r")
class_index = int(f.readlines()[1].strip().split(':')[1])
y_actual = all_classes[class_index-1]

# Study of frames
imgpath = r"..\FramesFlows\mytest\ApplyLipstick\v_ApplyLipstick_g01_c01.avi\frames"
flowspath = r"..\FramesFlows\mytest\ApplyLipstick\v_ApplyLipstick_g01_c01.avi\flows"


allframes = os.listdir(imgpath)
allflows = os.listdir(flowspath)

#Sort frames sequentially
allframes.sort(key = lambda x: int(x.split('_')[1].split('.')[0]))

#Sort flows sequentially
xflows = list(filter(lambda k: k.split('_')[1] == 'x',allflows))
yflows = list(filter(lambda k: k.split('_')[1] == 'y',allflows))
xflows.sort(key = lambda x: int(x.split('_')[2].split('.')[0]))
yflows.sort(key = lambda x: int(x.split('_')[2].split('.')[0]))

#Stack up frames for one video and divide into segments
stack = list()
for frame in allframes:
    img = cv2.imread(os.path.join(imgpath,frame))
    grayimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    stack.append(grayimg)
videoFrames = np.asarray(stack)
print ("Frames' shape:",videoFrames.shape)
framesSegments = np.array_split(videoFrames,5)

#Stack up flows for one video and divide into segments
xstack = list()
ystack = list()
for xflow,yflow in zip(xflows,yflows):
    xfl = cv2.imread(os.path.join(flowspath,xflow))
    yfl = cv2.imread(os.path.join(flowspath,yflow))
    grayxfl = cv2.cvtColor(xfl,cv2.COLOR_BGR2GRAY)
    grayyfl = cv2.cvtColor(yfl,cv2.COLOR_BGR2GRAY)
    xstack.append(grayxfl)
    ystack.append(grayyfl)
videoFlowsX = np.asarray(xstack)
videoFlowsY = np.asarray(ystack)
print ("Flows' shape:",videoFlowsX.shape)
flowsSegmentsX = np.array_split(videoFlowsX,5)
flowsSegmentsY = np.array_split(videoFlowsY,5)


Frames' shape: (149, 240, 320)
Flows' shape: (148, 240, 320)


In [113]:
#Get k(equal to segments) random samples from frames from segments
frameSamples = list()
for segment in framesSegments:
    frameSamples.append(segment[np.random.choice(segment.shape[0],1)])